In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from keras.layers import Dropout
import matplotlib.pyplot as plt

from google.colab import files
uploaded = files.upload()

Saving Crop_recommendation.csv to Crop_recommendation.csv


In [ ]:
df = pd.read_csv("Crop_recommendation.csv")
df.head()

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [ ]:
df.shape

(2200, 8)

In [ ]:
df.isna().sum()

N              0
P              0
K              0
temperature    0
humidity       0
ph             0
rainfall       0
label          0
dtype: int64

In [ ]:
df_mean = pd.pivot_table(df,index=['label'],aggfunc='mean')

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import random

phos_summary = df_mean.sort_values(by='P',ascending=False)

fig = make_subplots(rows=1, cols=2)
col_arr = ['blue', 'green', 'red', 'yellow']

top_10 = { 'y': phos_summary['P'][0:10].sort_values().index,
           'x': phos_summary['P'][0:10].sort_values()
         }

last_10 = { 'y': phos_summary['P'][-10:].sort_values().index,
           'x': phos_summary['P'][-10:].sort_values()
         }

fig.add_trace(
                go.Bar( top_10,
                       marker_color= random.choice(col_arr),
                       name = 'Most Phosporus Required',
                       orientation = 'h',
                       text = top_10['x']
                      ),
                row=1, col=1
             )

fig.add_trace(
                go.Bar( last_10,
                       marker_color= random.choice(col_arr),
                       name = 'Least Phosporus Required',
                       orientation = 'h',
                       text = last_10['x']
                      ),
                row=1, col=2
             )

fig.update_traces(texttemplate='%{text}',textposition='inside')
fig.update_layout(title='Phosporus',font_size=12)
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace( go.Bar( x = df_mean.index,
                       y = df_mean['N'],
                      marker_color = 'brown',
                      name = 'Nitrogen'
                     )
             )

fig.add_trace( go.Bar( x = df_mean.index,
                       y = df_mean['P'],
                      marker_color = 'red',
                      name = 'Phosphorous'
                     )
             )

fig.add_trace( go.Bar( x = df_mean.index,
                       y = df_mean['K'],
                      marker_color = 'yellow',
                      name = 'Potassium'
                     )
             )
fig.update_layout(title='Comparison between different compounds')


In [ ]:
import plotly.express as px

fig = px.scatter( x = df['temperature'],
                       y = df['rainfall'],
                      color = df['label'])
# fig.update_layout(title='Rainfall by temperature')
fig.show()

In [ ]:
Y_train = df['label']
X_train = df.drop('label', axis = 1)
print(X_train.head())
print("")
print(Y_train.head())

    N   P   K  temperature   humidity        ph    rainfall
0  90  42  43    20.879744  82.002744  6.502985  202.935536
1  85  58  41    21.770462  80.319644  7.038096  226.655537
2  60  55  44    23.004459  82.320763  7.840207  263.964248
3  74  35  40    26.491096  80.158363  6.980401  242.864034
4  78  42  42    20.130175  81.604873  7.628473  262.717340

0    rice
1    rice
2    rice
3    rice
4    rice
Name: label, dtype: object


In [ ]:
Encoder = OneHotEncoder(handle_unknown='ignore')

Y_train = pd.DataFrame(Encoder.fit_transform(df[['label']]).toarray())

print(Y_train)

       0    1    2    3    4    5    6    7    8    9   ...   12   13   14  \
0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
4     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
2195  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2196  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2197  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2198  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2199  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

       15   16   17   18   19   20   21  
0     0.0  0.0  0.0  

In [ ]:
sc = StandardScaler()
train_scaled = sc.fit_transform(X_train)
print(train_scaled)
data = train_scaled

[[ 1.0687974  -0.34455075 -0.1016875  ...  0.47266646  0.04330173
   1.8103605 ]
 [ 0.93332887  0.14061552 -0.14118477 ...  0.39705125  0.73487256
   2.24205791]
 [ 0.25598625  0.04964684 -0.08193887 ...  0.48695381  1.77151047
   2.92106603]
 ...
 [ 1.82742114 -0.61745677 -0.35841972 ... -0.19123516 -0.13812031
   1.27141766]
 [ 1.80032743 -0.64777967 -0.27942519 ... -0.86951801  0.37390383
   0.43154519]
 [ 1.44810927 -1.07230015 -0.35841972 ... -0.49802006  0.40109573
   0.68200518]]


In [ ]:
model = Sequential()
model.add(Dense(1000, activation='relu', input_shape=(data.shape[1],)))
model.add(Dropout(0.3))
model.add(Dense(500, activation='tanh'))
model.add(Dense(22, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train,Y_train, epochs = 100)

Epoch 1/100
69/69 [==============================] - 2s 12ms/step - loss: 1.4073 - accuracy: 0.5595
Epoch 2/100
69/69 [==============================] - 1s 11ms/step - loss: 0.5743 - accuracy: 0.8136
Epoch 3/100
69/69 [==============================] - 1s 12ms/step - loss: 0.3539 - accuracy: 0.8855
Epoch 4/100
69/69 [==============================] - 1s 17ms/step - loss: 0.2664 - accuracy: 0.9136
Epoch 5/100
69/69 [==============================] - 1s 11ms/step - loss: 0.2217 - accuracy: 0.9214
Epoch 6/100
69/69 [==============================] - 1s 12ms/step - loss: 0.2223 - accuracy: 0.9259
Epoch 7/100
69/69 [==============================] - 1s 12ms/step - loss: 0.1781 - accuracy: 0.9359
Epoch 8/100
69/69 [==============================] - 1s 12ms/step - loss: 0.1654 - accuracy: 0.9400
Epoch 9/100
69/69 [==============================] - 1s 11ms/step - loss: 0.1584 - accuracy: 0.9427
Epoch 10/100
69/69 [==============================] - 1s 11ms/step - loss: 0.1630 - accuracy: 0.9391